**整理数据**

In [1]:
import pandas as pd
from pathlib import Path
import json
import jieba
results = []

for fp in Path("/Users/wanghuan/Projects/simple-stt/outputs").glob("*.json"):
    with open(fp, "r") as f:
        d = json.load(f)
        
    text = "".join([r["text"] for r in d])
    results.append([fp.stem, text])

actuals = pd.read_excel("/Users/wanghuan/Projects/simple-stt/data-16k/groundtruth.xlsx")
actuals = actuals.rename(columns = {'filename':'fname'}).copy()
actuals['fname'] = actuals['fname'].str.replace(".pcm", "").copy()
    
preds = pd.DataFrame(results, columns = ['fname','pred']).copy()
merged = actuals.merge(preds, on='fname', how='inner')

def remove_punctuation(text):
    text = str(text)
    import re
    return re.sub(r'[^\w\s]', '', text)

def jieba_cut(text):
    text = str(text)
    return " ".join(jieba.cut(text))

merged['pred'] = merged['pred'].apply(remove_punctuation)
merged['target'] = merged['target'].apply(remove_punctuation)
merged['pred-cut'] = merged['pred'].apply(jieba_cut)
merged['target-cut'] = merged['target'].apply(jieba_cut)

/var/folders/9z/tc5kc_854l56plrg75dhnr7r0000gn/T/ipykernel_18240/2492553865.py:16: FutureWarning: The default value of regex will change from True to False in a future version.
  actuals['fname'] = actuals['fname'].str.replace(".pcm", "").copy()
Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/9z/tc5kc_854l56plrg75dhnr7r0000gn/T/jieba.cache
Loading model cost 0.251 seconds.
Prefix dict has been built successfully.


**计算指标**

In [2]:
import jiwer
import numpy as np
wers = []
cers = []
for i, row in merged.iterrows():
    try:
        wer = jiwer.wer(row['pred-cut'], row['target-cut'])
    except:
        wer = 1
    wers.append(wer)
    try:
        cer = jiwer.cer(row['pred'], row['target'])
    except:
        cer = 1
    cers.append(cer)
    

merged['wer'] = wers
merged['cer'] = cers
print(f"Average WER: {np.mean(wers)}")
print(f"Average CER: {np.mean(cers)}")

Average WER: 0.37600601270545303
Average CER: 0.3660218967923564


**Bad Case分析**

In [116]:
from IPython.display import Audio, display
import soundfile
badcase = merged[merged.wer > 0.7].fname.sample(1).iloc[0]
display(Audio(f"../data-16k/{badcase}.wav",autoplay=True,rate=16000))
record = merged[merged.fname == badcase]
print("Ground Truth:", record.iloc[0].target)
print("Prediction:", record.iloc[0].pred)
print("WER:", record.iloc[0].wer)
print("CER:", record.iloc[0].cer)


Ground Truth: 是
Prediction: 你你的呀是你是干什么喂hello喂喂喂不需要就是这个吗咦啊你是哪位啊是个意思吗嗯你士不好意思啊
WER: 0.967741935483871
CER: 0.9787234042553191


In [99]:
merged[merged.wer > 0.7]

,fname,target,pred,pred-cut,target-cut,wer,cer
0,log.20250304_audio_0,是有,不觉得方面是还有有这个不是,不 觉得 方面 是 还有 有 这个 不是,是 有,0.750000,0.846154
11,log.20250304_audio_19,啊你好嗯对10000切换,喂你好打错了对你个电话,喂 你好 打错 了 对 你 个 电话,啊 你好 嗯 对 10000 切换,0.750000,0.909091
17,log.20250304_audio_24,你好你是谁没有接着你是谁呀哦嗯之前什么奇怪啊业务你之前什么问题啊之前的什么问题呀,你好就可以形成一咱们还是不知道你好这空电话你是谁还是没有学没有这要啊嗯接着呢你是谁呀生成了吗...,你好 就 可以 形成 一 咱们 还是 不 知道 你好 这空 电话 你 是 谁 还是 没有 学...,你好 你 是 谁 没有 接着 你 是 谁 呀 哦 嗯 之前 什么 奇怪 啊 业务 你 之前 ...,0.703704,0.691667
19,log.20250304_audio_26,无,不用在,不用 在,无,1.000000,1.000000
25,log.20250304_audio_31,嗯嗯对嗯有是的嗯,来屏喂你好嗯简单问把这些的嗯对对对他的啊嗯你好是吧电话,来屏 喂 你好 嗯 简单 问 把 这些 的 嗯 对 对 对 他 的 啊 嗯 你好 是 吧 电话,嗯 嗯 对 嗯 有 是 的 嗯,0.761905,0.814815
28,log.20250304_audio_34,你好嗯你哪位嗯我是有的10万全款,你好嗯您哪位啊明白了消毒柜有什么嗯我是我应该说他们之前是你们你们聊过那个什么双留下带子什么嗯...,你好 嗯 您 哪位 啊 明白 了 消毒柜 有 什么 嗯 我 是 我 应该 说 他们 之前 是...,你好 嗯 你 哪位 嗯 我 是 有 的 10 万全 款,0.876923,0.886598
29,log.20250304_audio_35,是,你你的呀是你是干什么喂hello喂喂喂不需要就是这个吗咦啊你是哪位啊是个意思吗嗯你士不好意思啊,你 你 的 呀 是 你 是 干什么 喂 hello 喂喂 喂 不 需要 就是 这个 吗 咦 ...,是,0.967742,0.978723
42,log.20250304_audio_6,喂喂喂正忙着呢打电话呢好吧喂正打电话呢拜拜,我一直你你好多了呀喂如果不用我回呃正忙着呢打电话呢好吧喂不好意思给你打电话呢行嗯时没有那我们...,我 一直 你 你 好多 了 呀 喂 如果 不用 我 回 呃 正忙着 呢 打电话 呢 好 吧 ...,喂喂 喂 正忙着 呢 打电话 呢 好 吧 喂 正 打电话 呢 拜拜,0.738095,0.709677
50,log.20250305_audio_12,嗯你说有啊,,,嗯 你 说 有 啊,1.000000,1.000000
53,log.20250305_audio_15,喂方便方便怎么了,嗯这里为他没在干嘛呢在嘛啊没有搞得我所以我天的这个方便方便怎么了,嗯 这里 为 他 没 在 干嘛 呢 在 嘛 啊 没有 搞 得 我 所以 我天 的 这个 方便...,喂 方便 方便 怎么 了,0.826087,0.781250


In [104]:
merged[merged.wer > 0.7].target.str.len().mean()

8.130434782608695

In [105]:
merged[merged.wer < 0.3].target.str.len().mean()

18.28125

**Good Case分析**

In [117]:
merged[merged.wer < 0.3]

,fname,target,pred,pred-cut,target-cut,wer,cer
3,log.20250304_audio_11,喂对打错了吧,喂对打错了,喂 对 打错 了,喂 对 打错 了 吧,0.250000,0.200000
5,log.20250304_audio_13,哎你好嗯啊是的您是哪位呃正在考虑是的嗯可以你们贷款利率是多少你能贷多少钱就是您能贷多少钱,哎你好嗯啊是的您临时那边呃正在考虑是的呃可以你们贷款利率是多少你能贷多少钱就你能贷多少钱,哎 你好 嗯 啊 是 的 您 临时 那边 呃 正在 考虑 是 的 呃 可以 你们 贷款 利率...,哎 你好 嗯 啊 是 的 您 是 哪位 呃 正在 考虑 是 的 嗯 可以 你们 贷款 利率 ...,0.156250,0.159091
6,log.20250304_audio_14,通话已转至语音留言你尝试联系的用户无法接通请在提示音后录制留言录音完成后挂断即可,通话已短至语音留言你尝试联系的用户无法接通请在请在提示音后录制留言录音完成后挂断即可,通话 已 短 至 语音 留言 你 尝试 联系 的 用户 无法 接通 请 在 请 在 提示音 ...,通话 已 转至 语音 留言 你 尝试 联系 的 用户 无法 接通 请 在 提示音 后 录制 ...,0.153846,0.071429
7,log.20250304_audio_15,通话已转至语音留言你尝试联系的用户无法接通请在提示音后录制留言录音完成后挂断即可,通话已短至语音留言你尝试联系的用户无法接通并请在提示音后录制留言录音录音完成后挂断即可,通话 已 短 至 语音 留言 你 尝试 联系 的 用户 无法 接通 并 请 在 提示音 后 ...,通话 已 转至 语音 留言 你 尝试 联系 的 用户 无法 接通 请 在 提示音 后 录制 ...,0.153846,0.093023
20,log.20250304_audio_27,Hello你是傻子吗嗯我是啊,hello你是傻子吗啊我是啊,hello 你 是 傻子 吗 啊 我 是 啊,Hello 你 是 傻子 吗 嗯 我 是 啊,0.222222,0.142857
22,log.20250304_audio_29,喂哪里呀你有什么事吗啊不是不是没有兴趣没有没有没有,喂哪里呀有什么事吗是嗯啊不是不是没有兴趣三十几分钟没有没有没有,喂 哪里 呀 有 什么 事 吗 是 嗯 啊 不是 不是 没有 兴趣 三 十几分钟 没有 没有 没有,喂 哪里 呀 你 有 什么 事 吗 啊 不是 不是 没有 兴趣 没有 没有 没有,0.263158,0.258065
26,log.20250304_audio_32,喂喂你好你好你好你好你好你好你好你好你好你好你好你好你好你好你好喂喂喂喂喂喂喂喂喂喂喂喂喂喂...,喂喂你好你好你好你好你好你好你好你好你好你好你好你好你好你好行喂喂喂喂喂喂喂喂喂喂喂喂喂喂喂...,喂喂 你好 你好 你好 你好 你好 你好 你好 你好 你好 你好 你好 你好 你好 你好 行...,喂喂 你好 你好 你好 你好 你好 你好 你好 你好 你好 你好 你好 你好 你好 你好 你...,0.058824,0.036145
30,log.20250304_audio_36,嗯嗯喂你谁呀喂你谁喂喂喂你你谁呀嗯嗯喂喂喂啊你是哪位啊哎你是哪位你是哪位你先别问我需要多少资...,嗯嗯喂女士呀喂你喂喂喂喂喂你你谁呀嗯嗯喂喂喂喂谁呀喂啊你是哪位啊嗯你是哪位你是哪位你先别问我...,嗯 嗯 喂 女士 呀 喂 你 喂喂 喂喂 喂 你 你 谁 呀 嗯 嗯 喂喂 喂喂 谁 呀 喂...,嗯 嗯 喂 你 谁 呀 喂 你 谁 喂喂 喂 你 你 谁 呀 嗯 嗯 喂喂 喂 啊 你 是 ...,0.244898,0.261538
32,log.20250304_audio_38,哎你好呃是的有的你们能能能借多少钱呢我需要100万呃全款的,哎您好呃是的有的你们能能能借多少钱呢我需要一百万呃全款的,哎 您好 呃 是 的 有 的 你们 能能 能 借 多少 钱 呢 我 需要 一百万 呃 全款 的,哎 你好 呃 是 的 有 的 你们 能能 能 借 多少 钱 呢 我 需要 100 万 呃 全款 的,0.150000,0.142857
34,log.20250304_audio_4,你好你是谁呀有啊那你那你这边能贷多少呢您看有几种方式呢,你好你是谁呀有啊那你那你这边能贷多少呢你那有几种方式呢,你好 你 是 谁 呀 有 啊 那 你 那 你 这边 能贷 多少 呢 你 那 有 几种 方式 呢,你好 你 是 谁 呀 有 啊 那 你 那 你 这边 能贷 多少 呢 您 看 有 几种 方式 呢,0.095238,0.074074


In [95]:
goodcase = merged[merged.wer < 0.3].fname.sample(1).iloc[0]
display(Audio(f"../data-16k/{goodcase}.wav",autoplay=True,rate=16000))
record = merged[merged.fname == goodcase]
print("Ground Truth:", record.iloc[0].target)
print("Prediction:", record.iloc[0].pred)
print("WER:", record.iloc[0].wer)
print("CER:", record.iloc[0].cer)

Ground Truth: 嗯你说有啊
Prediction: 嗯你说有啊
WER: 0.0
CER: 0.0
